In [10]:
import itertools
import math
import string
from copy import deepcopy

import blosum as bl
import numpy as np
from enum import IntEnum

In [11]:
class Score(IntEnum):
    MATCH = 1
    MISMATCH = -1
    GAP = -1

# Assigning the constant values for the traceback
class Trace(IntEnum):
    STOP = 0
    LEFT = 1
    UP = 2
    DIAGONAL = 3

# Implementing the Smith Waterman local alignment
def smith_waterman(seq1, seq2):
    # Generating the empty matrices for storing scores and tracing
    row = len(seq1) + 1
    col = len(seq2) + 1
    matrix = np.zeros(shape=(row, col))
    tracing_matrix = np.zeros(shape=(row, col))

    # Initialising the variables to find the highest scoring cell
    max_score = -1
    max_index = (-1, -1)

    # Calculating the scores for all cells in the matrix
    for i in range(1, row):
        for j in range(1, col):
            # Calculating the diagonal score (match score)
            match_value = Score.MATCH if seq1[i - 1] == seq2[j - 1] else Score.MISMATCH
            diagonal_score = matrix[i - 1, j - 1] + match_value

            # Calculating the vertical gap score
            vertical_score = matrix[i - 1, j] + Score.GAP

            # Calculating the horizontal gap score
            horizontal_score = matrix[i, j - 1] + Score.GAP

            # Taking the highest score
            matrix[i, j] = max(0, diagonal_score, vertical_score, horizontal_score)

            # Tracking where the cell's value is coming from
            if matrix[i, j] == 0:
                tracing_matrix[i, j] = Trace.STOP

            elif matrix[i, j] == horizontal_score:
                tracing_matrix[i, j] = Trace.LEFT

            elif matrix[i, j] == vertical_score:
                tracing_matrix[i, j] = Trace.UP

            elif matrix[i, j] == diagonal_score:
                tracing_matrix[i, j] = Trace.DIAGONAL

            # Tracking the cell with the maximum score
            if matrix[i, j] >= max_score:
                max_index = (i,j)
                max_score = matrix[i, j]

    # Initialising the variables for tracing
    aligned_seq1 = ""
    aligned_seq2 = ""
    current_aligned_seq1 = ""
    current_aligned_seq2 = ""
    (max_i, max_j) = max_index

    # Tracing and computing the pathway with the local alignment
    while tracing_matrix[max_i, max_j] != Trace.STOP:
        if tracing_matrix[max_i, max_j] == Trace.DIAGONAL:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = seq2[max_j - 1]
            max_i = max_i - 1
            max_j = max_j - 1

        elif tracing_matrix[max_i, max_j] == Trace.UP:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = '-'
            max_i = max_i - 1

        elif tracing_matrix[max_i, max_j] == Trace.LEFT:
            current_aligned_seq1 = '-'
            current_aligned_seq2 = seq2[max_j - 1]
            max_j = max_j - 1

        aligned_seq1 = aligned_seq1 + current_aligned_seq1
        aligned_seq2 = aligned_seq2 + current_aligned_seq2

    # Reversing the order of the sequences
    aligned_seq1 = aligned_seq1[::-1]
    aligned_seq2 = aligned_seq2[::-1]

    return aligned_seq1, aligned_seq2, max_score

Вход: слово длины 4 по дефолту но с изменением, нуклеотидные последовательности,
ВЫход: упорядоченный список диагоналей с наибольшим числом совпадений
сравнение индексов (длина слова, последовательности)

In [12]:
def to_quadriple(str):
    sum = 0.0
    cnt = 0
    for s in reversed(str):
        if s == 'A':
            sum += math.pow(4, cnt) * 0
        elif s == 'C':
            sum += math.pow(4, cnt) * 1
        elif s == 'G':
            sum += math.pow(4, cnt) * 2
        elif s == 'T':
            sum += math.pow(4, cnt) * 3
        cnt += 1
    return sum


In [13]:
def to_map(seq: string, k: int):
    dict = {}
    for i in range(len(seq) - k + 1):
        key = seq[i : i + k]
        if dict.get(key) is None:
            dict[key] = []
        dict[key].append(i)
    return dict

In [14]:
def sub_maps_diff_pos(map_1, map_2):
    dist = {}
    for k1 in map_1.keys(): #k1 - 'AA'; val1 - [1, 2, 3]
        if map_2.get(k1) is None:
            continue

        for pos1 in map_1[k1]:
            for pos2 in map_2[k1]:
                delta = pos2 - pos1
                if dist.get(delta) is None:
                    dist[delta] = []
                dist[delta].append(pos1)


    res = []
    for gap in dist:
        li = dist[gap]
        res.append(Diag(convert_diag_to_xy(gap, li)))

    return res

def cutt_off(dist, count=1):
    res = {}
    for key in dist.keys():
        if len(dist[key]) > count:
            res[key] = dist[key]
    return res


In [15]:
def sub_maps(map_1, map_2):
    # dict[subsequence] = set(distance(subsequence))
    dist = {}

    for k1 in map_1.keys():
        if map_2.get(k1) is None:
            continue
        if dist.get(k1) is None:
            dist[k1] = []
        for pos1 in map_1[k1]:
            for pos2 in map_2[k1]:
                dist[k1].append(pos2 - pos1)
    return dist

In [16]:
def convert_to_matrix(diffs, n1, n2):
    res = np.zeros((n1, n2))
    for gap in diffs:
        for i, pos1 in enumerate(diffs[gap]):
            pos2 = pos1 + gap
            res[pos1][pos2] = 1
    return res


In [67]:
s1 = 'GCATCGGC'
s2 = 'CCATCGCCATCG'

m1 = to_map(s1, 2)
m2 = to_map(s2, 2)
# diff = sub_maps(m1, m2)
diff2 = sub_maps_diff_pos(m1, m2)
# print(m1)
# print(m2)
# print(diff)
print(diff2)
# print(convert_to_matrix(diff2, len(s1), len(s2)))


[<__main__.Diag object at 0x7f2d84e017b0>, <__main__.Diag object at 0x7f2d84e029b0>, <__main__.Diag object at 0x7f2d84e03a90>, <__main__.Diag object at 0x7f2d84e039d0>]


In [80]:
class Diag:
    # def __init__(self, gap, poss1):
    #     MAX = float('inf')
    #     MIN = float('-inf')
    #     x_min = MAX
    #     x_max = MIN
    #     y_min = MAX
    #     y_max = MIN
    #     for i, pos1 in enumerate(poss1):
    #         pos2 = pos1 + gap
    #         x_min = min(x_min, pos1)
    #         x_max = max(x_max, pos1)
    #         y_min = min(y_min, pos2)
    #         y_max = max(y_max, pos2)
    #     self.x_min = x_min
    #     self.x_max = x_max
    #     self.y_min = y_min
    #     self.y_max = y_max

    def is_straight(self):
        return (self.y_max - self.y_min) == (self.x_max - self.x_min)

    def __init__(self, li):
        self.x_min = li[0]
        self.y_min = li[1]
        self.x_max = li[2]
        self.y_max = li[3]

    def __add__(self, diag):
        return Diag([min(self.x_min, diag.x_min), min(self.y_min, diag.y_min), max(diag.x_max, self.x_max), max(diag.y_max, self.y_max)])

    def __len__(self):
        return self.x_max - self.x_min + self.y_max - self.y_min

    def __getitem__(self, item):
        x = self.x_min + item
        y = self.y_min + item
        if x > self.x_max or y > self.y_max:
            raise KeyError
        return Diag([x, y, x, y])

    def __eq__(self, other):
        return self.x_min == other.x_min and self.y_min == other.y_min and self.x_max == other.x_max and self.y_max == other.y_max

    def __le__(self, other):
        return len(self) <= len(other)

    def __lt__(self, other):
        return len(self) < len(other)

    def __hash__(self):
        return hash(repr(self))

    def __str__(self):
        return f"({self.x_min}, {self.y_min}) -> ({self.x_max}, {self.y_max}): " + ("straight" if self.is_straight() else "not straight")

In [68]:
def convert_diag_to_xy(gap, poss1):
    MAX = float('inf')
    MIN = float('-inf')
    x_min = MAX
    x_max = MIN
    y_min = MAX
    y_max = MIN
    for i, pos1 in enumerate(poss1):
        pos2 = pos1 + gap
        x_min = min(x_min, pos1)
        x_max = max(x_max, pos1)
        y_min = min(y_min, pos2)
        y_max = max(y_max, pos2)
    return x_min, y_min, x_max, y_max


In [119]:
def find_best_for_diag(diag_scores_old, gap):
    concatted = False
    diag_scores_new = {}
    diag_scores = deepcopy(diag_scores_old)
    for d1 in diag_scores:
        MAX = float('-inf')
        best_d2 = None
        for d2 in diag_scores:
            if not d1.__eq__(d2) and d2.x_min >= d1.x_min and d2.y_min >= d1.y_min:
                # чтобы вторая диагоналей была не выше и не левее первой
                if d2.x_min  <= d1.x_max:
                    dist = d2.y_min - d1.y_min
                elif d2.y_min <= d1.y_max:
                    dist = d2.x_min - d1.x_min
                else:
                    dist = d2.y_min - d1.y_max + d2.x_min - d1.x_max
                res = diag_scores[d1] + diag_scores[d2] + dist * gap
                if res > diag_scores[d1] and res > diag_scores[d2] and res > MAX:
                    MAX = res
                    best_d2 = d2

        if best_d2 is not None:
            diag_scores_new[best_d2] = float('-inf')
            diag_scores_new[d1] = float('-inf')
            concatted = True
            diag_scores_new[(d1 + best_d2)] = MAX
        else:
            diag_scores_new[d1] = diag_scores[d1]
    return concatted, diag_scores_new

In [121]:
def get_best_diagonals_concatenation_with_scores(str1, str2, substr_len=2, gap=-10, blosum_cf=5):
    map_1 = to_map(str1, substr_len)
    map_2 = to_map(str2, substr_len)
    diags = get_top_10(sub_maps_diff_pos(map_1, map_2))
    diffs2 = []
    blosum_matrix = bl.BLOSUM(62)
    diag_scores = {}

    for d1 in diags:
        diag_scores[d1] = 0.0

        for i in range(d1.x_max - d1.x_min):
            len_1_diag = d1[i]
            diag_scores[d1] += blosum_matrix[str1[len_1_diag.x_min] + str2[len_1_diag.y_min]]
        if diag_scores[d1] < blosum_cf:
            del diag_scores[d1]
        else:
            diffs2.append(d1)


    concatted = True
    while concatted:
        concatted, diag_scores = find_best_for_diag(diag_scores, gap)



    res = {}
    for diag in diag_scores:
        score = diag_scores[diag]
        if score > 0:
            res[diag] = score
    return res


s3 = 'MALRKGGLALALLLLSWVALGPRSLEGADPGTPGEAEGPACPAACVCSYDDDADELSVFCSSRNLTRLPDGVPGGTQALW\
LDGNNLSSVPPAAFQNLSSLGFLNLQGGQLGSLEPQALLGLENLCHLHLERNQLRSLALGTFAHTPALASLGLSNNRLSR\
LEDGLFEGLGSLWDLNLGWNSLAVLPDAAFRGLGSLRELVLAGNRLAYLQPALFSGLAELRELDLSRNALRAIKANVFVQ\
LPRLQKLYLDRNLIAAVAPGAFLGLKALRWLDLSHNRVAGLLEDTFPGLLGLRVLRLSHNAIASLRPRTFKDLHFLEELQ\
LGHNRIRQLAERSFEGLGQLEVLTLDHNQLQEVKAGAFLGLTNVAVMNLSGNCLRNLPEQVFRGLGKLHSLHLEGSCLGR\
IRPHTFTGLSGLRRLFLKDNGLVGIEEQSLWGLAELLELDLTSNQLTHLPHRLFQGLGKLEYLLLSRNRLAELPADALGP\
LQRAFWLDVSHNRLEALPNSLLAPLGRLRYLSLRNNSLRTFTPQPPGLERLWLEGNPWDCGCPLKALRDFALQNPSAVPR\
FVQAICEGDDCQPPAYTYNNITCASPPEVVGLDLRDLSEAHFAPC'

s4 = 'MALRKGGLALALLLLSWVALGPRSLEGADPGTPGEAEGPACPAACVCSYDDDADELSVFCSSRNLTRLPDGVPGGTYUIW\
LDGNNLSSVPPAAFQNLSSLGFLNLQGGQLGSLEPQALLGLENLCHLHLERNQLRSLALGTFAHTPALASLGLSNNRLSR\
LEDGLFEGLGSLWDLNLGWNSLAVLPDAAFRGLGSLRELVLUGNRLAYLQPALFSGLAELRELDLSRNALRAIKANVFVQ\
LPRLQKLYLDRNLIAAVAPGAFLGLKALRWLDLSHNRVAGLLEDTFPGLLGLRVLRLSHNAIASLRPRTFKDLHFLEELQ\
LGHNRIRQLAERSFEGLGQLEVLTLDHNQLQEVKAGAFLGLTNVAVMNLSGNCLRNLPEQVFRGLGKLHSLHLEGSCLGR\
IRPHTFTGBJJIBJNJKNKNKNVGIEEQSLWGLAELLELDLTSNQLTHLPHRLFQGLGKLEYLLLSRNRLAELPADALGP\
LQRAFWLDVSHNRLEALPNSLLAPLGRLRYLSLRNNSLRTFTPQPPGLERLWLEGNPWDCGCPLKAHGGDRYYVPR\
FVQAICEGDDCQPPAYTYNNITCASPPEVVGLDLRDLSEAHFAPC'

get_best_diagonals_concatenation_with_scores(s3, s4, substr_len=2, gap=-5)


(82, 202) -> (384, 504): straight
339.0
(80, 248) -> (373, 517): not straight
641.0
(111, 303) -> (373, 589): not straight
188.0
(80, 344) -> (267, 517): not straight
432.0
(62, 374) -> (195, 507): straight
94.0
(444, 108) -> (516, 180): straight
69.0
(108, 444) -> (180, 516): straight
69.0
(453, 93) -> (514, 154): straight
54.0
(516, 23) -> (525, 32): straight
7.0
(23, 516) -> (32, 525): straight
7.0
(133, 589) -> (141, 597): straight
9.0
(590, 290) -> (596, 296): straight
5.0
(590, 151) -> (596, 157): straight
5.0
(71, 553) -> (76, 558): straight
6.0
(591, 80) -> (596, 85): straight
7.0
(80, 587) -> (85, 592): straight
7.0
(462, 12) -> (464, 14): straight
8.0
(12, 462) -> (14, 464): straight
8.0
(18, 476) -> (20, 478): straight
8.0
(337, 107) -> (338, 108): straight
6.0
(107, 337) -> (108, 338): straight
6.0
(485, 79) -> (486, 80): straight
11.0
(79, 485) -> (80, 486): straight
11.0
(572, 89) -> (573, 90): straight
7.0
(89, 568) -> (90, 569): straight
7.0
(90, 41) -> (91, 42): straig

In [27]:
class Diagonal:
    def __init__(self, gap, pos1):
        self.pos1 = pos1
        self.pos2 = pos1 + gap
        self.gap = gap

    def concat(self, diag):
        pass

    def __str__(self):
        return f"{self.pos1}, {self.pos2}, gap={self.gap}"


def set_diagonals(arr):
    res = []
    res_s = []

    for i in arr:
        if len(res_s) == 0 or abs(res_s[-1] - i) <= 1:
            res_s.append(i)
        else:
            res.append(Diagonal(res_s[0], res_s[-1] - res_s[0]))
            res_s = [i]
    res.append(Diagonal(res_s[0], res_s[-1] - res_s[0]))


    return res

def get_top_10(dist):
    res = sorted(dist)
    return res[:10:-1]
    # for i in dist:
    #     temp.append(len(dist[i]))
    # temp = sorted(temp)
    # if len(temp) > 10:
    #     plank = temp[9]
    # else:
    #     plank = temp[0]
    # res = {}
    # for i in dist:
    #     if len(dist[i]) >= plank:
    #         res[i] = dist[i]
    # return res


In [ ]:
cutten = get_top_10(sub_maps_diff_pos(m1, m2))